In [2]:
import cv2  
import numpy as np
import matplotlib.pyplot as plt
import csv

  TAREA: Los ejemplos ilustrativos anteriores permiten saber el número de monedas presentes en la imagen. ¿Cómo saber la cantidad de dinero presente en ella? Sugerimos identificar de forma interactiva (por ejemplo haciendo clic en la imagen) una moneda de un valor determinado en la imagen (por ejemplo de 1€). Tras obtener esa información y las dimensiones en milímetros de las distintas monedas, realiza una propuesta para estimar la cantidad de dinero en la imagen. Muestra la cuenta de monedas y dinero sobre la imagen. No hay restricciones sobre utilizar medidas geométricas o de color. 

  Una vez resuelto el reto con la imagen ideal proporcionada, captura una o varias imágenes con monedas. Aplica el mismo esquema, tras identificar la moneda del valor determinado, calcula el dinero presente en la imagen. ¿Funciona correctamente? ¿Se observan problemas?

  Nota: Para establecer la correspondencia entre píxeles y milímetros, comentar que la moneda de un euro tiene un diámetro de 23.25 mm. la de 50 céntimos de 24.35, la de 20 céntimos de 22.25, etc. 

Extras: Considerar que la imagen pueda contener objetos que no son monedas y/o haya solape entre las monedas. Demo en vivo. 

In [16]:
# --- 1. Cargar imagen ---
img = cv2.imread("Monedas.jpg")
if img is None:
    raise IOError("No se pudo cargar la imagen 'Monedas.jpg'.")


img_display = img.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 5)

# --- 2. Detectar monedas con HoughCircles ---
circles = cv2.HoughCircles(
    gray, cv2.HOUGH_GRADIENT, dp=1, minDist=50,
    param1=100, param2=45, minRadius=25, maxRadius=100
)



if circles is None:
    raise ValueError("No se detectaron monedas en la imagen.")

circles = np.uint16(np.around(circles))

# Dibujar las monedas detectadas para referencia
for (x, y, r) in circles[0, :]:
    cv2.circle(img_display, (x, y), r, (0, 255, 0), 2)
    cv2.circle(img_display, (x, y), 2, (0, 0, 255), 3)

cv2.imshow("Monedas detectadas (haz clic en una de referencia)", img_display)

# --- 3. Variables globales para el clic ---
ref_diametro_mm = 23.25  # Moneda de referencia: 1€
escala_mm_px = None
ref_pos = None

# --- 4. Función para manejar el clic ---
def seleccionar_moneda(event, x, y, flags, param):
    global escala_mm_px, ref_pos

    if event == cv2.EVENT_LBUTTONDOWN:
        # Buscar el círculo más cercano al punto clicado
        distancias = [np.sqrt((x - cx)**2 + (y - cy)**2) for (cx, cy, _) in circles[0, :]]
        idx = np.argmin(distancias)
        (cx, cy, r) = circles[0][idx]
        ref_pos = (cx, cy)

        # Calcular escala (mm/px)
        diametro_ref_px = 2 * r
        escala_mm_px = ref_diametro_mm / diametro_ref_px

        print(f"Moneda de referencia seleccionada en ({cx}, {cy})")
        print(f"Radio en píxeles: {r:.2f}")
        print(f"Escala: {escala_mm_px:.5f} mm/px")

        cv2.destroyAllWindows()

# Asignar la función al clic del ratón
cv2.setMouseCallback("Monedas detectadas (haz clic en una de referencia)", seleccionar_moneda)

cv2.waitKey(0)

# --- 5. Si se ha seleccionado una referencia, clasificar y calcular ---
if escala_mm_px is None:
    raise ValueError("No se seleccionó ninguna moneda de referencia.")

# Diámetros reales de monedas (mm) y sus valores (€)
monedas_info = {
    16.25: 0.01,
    18.75: 0.02,
    19.75: 0.10,
    21.25: 0.05,
    22.25: 0.20,
    23.25: 1.00,
    24.25: 0.50,
    25.75: 2.00
}

total = 0
resultado = img.copy()

for (x, y, r) in circles[0, :]:
    diametro_mm = 2 * r * escala_mm_px
    # Buscar el diámetro más cercano
    diametro_real = min(monedas_info.keys(), key=lambda d: abs(d - diametro_mm))
    valor = monedas_info[diametro_real]
    total += valor

    # Dibujar en la imagen
    cv2.circle(resultado, (x, y), r, (0, 255, 0), 2)
    cv2.putText(resultado, f"{valor:.2f}EUR", (x - 25, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

# Mostrar el total
cv2.putText(resultado, f"Total: {total:.2f} EUR", (50, 50),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

cv2.imshow("Resultado final", resultado)
cv2.waitKey(0)
cv2.destroyAllWindows()


C:\Users\dasum\AppData\Local\Temp\ipykernel_14776\3293355993.py:42: RuntimeWarning: overflow encountered in scalar subtract
  distancias = [np.sqrt((x - cx)**2 + (y - cy)**2) for (cx, cy, _) in circles[0, :]]
C:\Users\dasum\AppData\Local\Temp\ipykernel_14776\3293355993.py:42: RuntimeWarning: overflow encountered in scalar add
  distancias = [np.sqrt((x - cx)**2 + (y - cy)**2) for (cx, cy, _) in circles[0, :]]


Moneda de referencia seleccionada en (136, 346)
Radio en píxeles: 92.00
Escala: 0.12636 mm/px


TAREA: La tarea consiste en extraer características (geométricas y/o visuales) de las tres imágenes completas de partida, y *aprender* patrones que permitan identificar las partículas en nuevas imágenes. Para ello se proporciona como imagen de test *MPs_test.jpg* y sus correpondientes anotaciones *MPs_test_bbs.csv* con la que deben obtener las métricas para su propuesta de clasificación de microplásticos, además de la matriz de confusión. La matriz de confusión permitirá mostrar para cada clase el número de muestras que se clasifican correctamente de dicha clase, y el número de muestras que se clasifican incorrectamente como perteneciente a una de las otras dos clases.

En el trabajo [SMACC: A System for Microplastics Automatic Counting and Classification](https://doi.org/10.1109/ACCESS.2020.2970498), las características geométricas utilizadas fueron:

- Área en píxeles
- Perímetro en píxeles
- Compacidad (relación entre el cuadrado del perímetro y el área de la partícula)
- Relación del área de la partícula con la del contenedor
- Relación del ancho y el alto del contenedor
- Relación entre los ejes de la elipse ajustada
- Definido el centroide, relación entre las distancias menor y mayor al contorno

Si no se quedan satisfechos con la segmentación obtenida, es el mundo real, también en el README comento técnicas recientes de segmentación, que podrían despertar su curiosidad.
